### 주의사항

text_mining_ver02에서 kobert와 transformer 의존성 충돌로 인해
문서 요약은 text_mining_ver01에서 수행하였음

## 문서 요약

문서요약은 주어진 문서가 담고 있는 중요한 내용을 요약하여 짧은 텍스트를 생성하는 작업을 말한다.

### 문서 요약의 이해
문서 요약은 추출 요약(Extractive Summarization)과  생성 요약(Abstractive Summarization)으로 구분된다.
추출 요약 : 원본 문서에 있는 주요 문장 혹은 단어를 추출하여 요약문을 작성한다.
문장의 중요도를 평가해 순위를 매긴 후, 상위의 문장을 선택한다.
전체 입력 텍스트와 코사인 유사도가 높은 문장을 정렬한 후에 상위 문장 2개로 요약문을 만든다.

생성 요약 : 원본 문서에 있지 않은 문장이나 단어를 생성해 요약문을 만들기 때문에 추출 요약에 비해 상대적으로 난이도가 높다.
원본 텍스트에 대한 이해를 전제로 하여 요약문을 새로 생성하기 때문에 Seq2Seq 모형을 사용한다

#### 문서 요약 성능 지표 : ROUGE

ROUGE : 모형에 의해 요약된 요약문을 전문가가 요약한 요약문, 즉 정답과 비교하여 성능을 측정한다.
단어의 출현 순서와 일치하는 정도를 정밀도과 재현율, F1Score로 측정한다.
즉, 전문가가 사용한 단어와 만들어낸 단어가 얼마나 일치하는지 보는 것이다.

#### 데이터셋
한국의 경우, AIHub에서 "요약문 및 레포트 생성 데이터"라는 추출요약문과 생성요약문을 함께 제공하고 있다.
문서 요약을 지원하는 트랜스포머 변형 모형으로 여러가지가 있는데, MBart-50은 한국어 요약을 제공한다.

#### 파이프라인을 이용한 문서 요약

In [6]:
from transformers import pipeline

# 문서요약을 위한 파이프라인 생성
summarizer = pipeline("summarization", framework="tf")
# 요약 대상 원문 - 텍스트마이닝의 정의(Wikipedia)
text = '''Text mining, also referred to as text data mining (abbr.: TDM), similar to text analytics,
        is the process of deriving high-quality information from text. It involves
        "the discovery by computer of new, previously unknown information,
        by automatically extracting information from different written resources."
        Written resources may include websites, books, emails, reviews, and articles.
        High-quality information is typically obtained by devising patterns and trends
        by means such as statistical pattern learning. According to Hotho et al. (2005)
        we can distinguish between three different perspectives of text mining:
        information extraction, data mining, and a KDD (Knowledge Discovery in Databases) process.'''
summary_text = summarizer(text) #파이프라인으로 문서요약 수행
print("요약문:\n", summary_text)
print("원문 길이:", len(text), "요약문 길이:", len(summary_text[0]["summary_text"]))

No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Your max_length is set to 200, but your input_length is only 146. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=73)


요약문:
 [{'summary_text': 'text data mining is the process of deriving high-quality information from text . it involves the discovery by computer of new, previously unknown information . a KDD (Knowledge Discovery in Databases) process is similar to text analytics .'}]
원문 길이: 771 요약문 길이: 239


사용할 모델을 정하지 않아 Summarization에 default로 "DistilBART"를 사용하였다.
DistilBART는 BART를 경량화한 버전이다.

#### T5 모형과 자동 클래스를 이용한 문서 요약

BERT와 GPT가 각각 트랜스포머의 인코더와 디코더만을 사용해서 모형을 만들어다면, T5는 인코더와 디코더를 모두 사용하여 다양한 자연어 처리 작업에 사용될 수 있는
통합 text-to-text 구조를 제안하였다.


In [5]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=512)
print("tokenizer type:", type(tokenizer))

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")
print("model type:", type(model))

tokenizer type: <class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


model type: <class 'transformers.models.t5.modeling_tf_t5.TFT5ForConditionalGeneration'>


In [8]:
preprocess_text = text.strip().replace("\n", "")
input_text = "summarize : " + preprocess_text

tokenized_text = tokenizer.encode(input_text, return_tensors = "tf")

In [9]:
summary_ids = model.generate(tokenized_text,
                             num_beams = 4, #가장 나올 확률이 높은 단어들 선정 -> 하지만, 더 좋은 확률을 가진 단어가 다음에 숨어있을 수도 있기 때문에 4개의 경우의 수를 저장한다.
                             no_repeat_ngram_size = 3, #한 번 "the dog is"가 만들어졌으면, 요약문에 다시 나올 수 없다.
                             min_length = 30,
                             max_length = 100,
                             early_stopping = True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True) #문장의 종류를 나타내는 토큰이 선택되면 생성을 종료할지 여부

print("Summarized text : \n", output)
print("Origin text length : ", len(text), "Summarized text length : ", len(output))

Summarized text : 
 text data mining is the process of deriving high-quality information from text. it involves the discovery by computer of new, previously unknown information. a KDD (Knowledge Discovery in Databases) process is similar to text analytics.
Origin text length :  771 Summarized text length :  236


In [11]:
input_text = "translate English to German: That is good"

tokenized_text = tokenizer.encode(input_text, return_tensors="tf")

result = model.generate(tokenized_text,
                        num_beams=4,
                        no_repeat_ngram_size=3,
                        max_length=100,
                        early_stopping=True)

output = tokenizer.decode(result[0], skip_special_tokens=True)
print("Translated text:\n", output)

Translated text:
 Das ist gut.


#### T5 모형과 트레이너를 이용한 미세조정학습

In [12]:
import tensorflow as tf
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5TokenizerFast.from_pretrained("t5-small", model_max_length = 1024)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [13]:
text = '''The Inflation Reduction Act lowers prescription drug costs, health care costs,
and energy costs. It's the most aggressive action on tackling the climate crisis in American history,
which will lift up American workers and create good-paying, union jobs across the country.
It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share.
And no one making under $400,000 per year will pay a penny more in taxes.'''

preprocess_text = text.strip().replace("\n", "")
input_text = "Summarize : "+preprocess_text

preprocessed_text = tokenizer.encode(input_text, return_tensors="tf")
summary_ids = model.generate(tokenized_text,
                             num_beams = 4,
                             no_repeat_ngram_size = 3,
                             min_length = 30,
                             max_length = 100,
                             early_stopping = True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens = True)

print("Summarized text : \n", output)
print("Original text length : ", len(text), "Summarized text length : ", len(output))

Summarized text : 
 Das ist gut so, wie es der Fall ist, d. h. daß dies nicht der Fall sein wird.
Original text length :  441 Summarized text length :  77


In [14]:
from datasets import load_dataset
billsum = load_dataset("billsum", split = "ca_test")
billsum = billsum.train_test_split(test_size = 0.2)
example = billsum["train"][0]
print("Billsum의 데이터 예 - 첫 항목")
print("\tText:", example['text'][:50])
print("\tSummary:", example['summary'][:50])
print("\tTitle:", example['title'][:50])

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Billsum의 데이터 예 - 첫 항목
	Text: The people of the State of California do enact as 
	Summary: Existing law establishes the California Homebuyer’
	Title: An act to amend Section 51504 of the Health and Sa


In [15]:
def preprocess_text(data):
    inputs = ["Summarize : " + doc for doc in data["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(data["summary"], max_length = 128, truncation = True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_text, batched = True, remove_columns=billsum["train"].column_names)
tokenized_billsum

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

In [16]:
#디코더에 필요한 라벨 입력을 자동으로 생성하는 역할
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model)

In [18]:
#rouge를 통한 성능 평가
import numpy as np
import evaluate
rouge = evaluate.load("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # 생성한 요약 토큰을 텍스트로 디코드
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # 라벨에서 디코드할 수 없는 -100을 교체
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # 라벨을 텍스트로 디코드 -> 전문가가 생성한 정답
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # 디코드된 요약문과 라벨로 ROUGE 스코어 계산
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
##미세조정 -> 이건 파이토치로만 가능

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./summary",         # 모형 예측과 체크포인트 저장 폴더
    evaluation_strategy="epoch",    # 평가 단위, 여기서는 epoch를 선택
    learning_rate=2e-5,             # 학습률
    per_device_train_batch_size=16, # 학습에 사용할 배치 크기
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 크기
    weight_decay=0.01,              # 가중치 감쇠 값
    save_total_limit=3,             # 저장할 체크포인트의 최대값
    num_train_epochs=4,             # 에포크 수
    predict_with_generate=True,     # 평가지표(ROUGE) 계산을 위해 generate할 지의 여부 -> compute metrice를 사용할 것인지
)

trainer = Seq2SeqTrainer(
    tokenizer=tokenizer,
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: 
Seq2SeqTrainer requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFSeq2SeqTrainer".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


#### 한글 문서 요약

In [1]:
text = """디아블로는 액션 롤플레잉 핵 앤드 슬래시 비디오 게임이다.
플레이어는 주변 환경을 마우스로 사용해 영웅을 움직이게 한다.
주문을 외는 등의 다른 활동은 키보드 입력으로 이루어진다.
플레이어는 이 게임에서 장비를 획득하고, 주문을 배우고, 적을 쓰러뜨리며, NPC와 대화를 나눌 수 있다.
지하 미궁은 주어진 형식이 있고 부분적으로 반복되는 형태가 존재하나 전체적으로 보면 무작위로 생성된다.
예를 들어 지하 묘지의 경우에는 긴 복도와 닫힌 문들이 존재하고, 동굴은 좀 더 선형 형태를 띠고 있다.
플레이어에게는 몇몇 단계에서 무작위의 퀘스트를 받는다.
이 퀘스트는 선택적인 사항이나 플레이어의 영웅들을 성장시키거나 줄거리를 이해하는데 도움을 준다.
그러나 맨 뒤에 두 퀘스트는 게임을 끝내기 위해 완료시켜야 한다."""

preprocess_text = text.strip().replace("\n","")

In [5]:
from transformers import PreTrainedTokenizerFast, TFBartForConditionalGeneration
import tensorflow as tf

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = TFBartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

# 전처리된 텍스트 토큰화
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="tf")

# 요약 생성
summary_ids = model.generate(tokenized_text,
                             num_beams=4,  # beam의 길이
                             no_repeat_ngram_size=3,  # 동어 반복을 피하기 위해 사용
                             min_length=10,  # 요약문의 최소 토큰 수
                             max_length=150,  # 요약문의 최대 토큰 수
                             early_stopping=True)  # EOS 토큰을 만나면 종료

# 요약문 디코딩
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


디아블로는 액션 롤플레잉 핵 앤드 슬래시 비디오 게임이다.플레이어는 주변 환경을 마우스로 사용해 영웅을 움직이게 한다.주문을 외는 등의 다른 활동은 키보드 입력으로 이루어진다.


In [7]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = TFAutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", from_pt=True)

# 전처리된 텍스트를 토큰화하고 TensorFlow 텐서로 변환
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="tf")

# 요약 생성
summary_ids = model.generate(tokenized_text,
                             num_beams=4,  # beam의 길이
                             no_repeat_ngram_size=2,  # 동어 반복을 피하기 위해 사용
                             min_length=10,  # 요약문의 최소 토큰 수
                             max_length=150,  # 요약문의 최대 토큰 수
                             early_stopping=True)  # EOS 토큰을 만나면 종료

# 요약문 디코딩
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

Loading a PyTorch model in TensorFlow, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.


ModuleNotFoundError: No module named 'torch'